In [1]:
import os
import nibabel as nib
import numpy as np


In [5]:
def nifti_to_npy(nifti_file):
    """
    Convert a NIfTI file to a numpy array.
    """
    img = nib.load(nifti_file)
    data = img.get_fdata()
    return np.array(data)

def process_patient_folders(base_dir):
    """
    Process each patient folder to convert NIfTI files to numpy arrays
    and organize them into the specified folder structure.
    """
    for patient_no in range(101, 151):  # Loop from patient001 to patient100
        patient_folder_name = f"patient{str(patient_no).zfill(3)}"  # patient001, patient002, ..., patient100
        patient_dir = os.path.join(base_dir, 'testing', patient_folder_name)
        target_dir = os.path.join(base_dir, 'testprocessed', f"patient{patient_no}")  # Output folder
        
        # Create directories for images and masks within the patient's folder
        images_dir = os.path.join(target_dir, 'image')
        masks_dir = os.path.join(target_dir, 'mask')
        os.makedirs(images_dir, exist_ok=True)
        os.makedirs(masks_dir, exist_ok=True)
        
        # Loop through each NIfTI file in the patient's folder and convert to numpy array
        for file in os.listdir(patient_dir):
            if file.endswith(".nii.gz"):
                nifti_path = os.path.join(patient_dir, file)
                npy_array = nifti_to_npy(nifti_path)
                
                # Determine if it's an image or a mask based on the filename
                if '_gt' in file:
                    npy_target_path = os.path.join(masks_dir, file.replace('.nii.gz', '.npy'))
                else:
                    npy_target_path = os.path.join(images_dir, file.replace('.nii.gz', '.npy'))
                
                # Convert 3D to 2D: Assuming we want to save slices along a specific axis (e.g., axis=2)
                for slice_idx in range(npy_array.shape[2]):
                    slice_npy = npy_array[:, :, slice_idx]
                    slice_npy_path = npy_target_path.replace('.npy', f'_slice{slice_idx}.npy')
                    np.save(slice_npy_path, slice_npy)
                    

In [6]:
base_dir = 'D:\Kodingan\Kodingan TA\database'  # Modify this path to your base directory
process_patient_folders(base_dir)

In [2]:
def delete_4d_npy_files_recursive(directory):
    """
    Recursively delete all .npy files containing '_4d' in their filenames within the specified directory and its subdirectories.
    """
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.npy') and '_4d' in file:
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")

In [ ]:
directory = 'D:\Kodingan\Kodingan TA\database\\testprocessed'  # Modify this to your target directory
delete_4d_npy_files_recursive(directory)

In [3]:
import matplotlib.pyplot as plt

def load_npy_files(directory):
    """
    Load all .npy files from the specified directory and return the arrays along with their filenames.
    """
    files = sorted([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.npy')])
    arrays = [np.load(f) for f in files]
    filenames = [os.path.basename(f) for f in files]  # Extract filenames
    return arrays, filenames

def display_images_and_masks(patient_dir):
    """
    Display all image slices and their corresponding masks for a given patient, including the filenames.
    """
    images_dir = os.path.join(patient_dir, 'image')
    masks_dir = os.path.join(patient_dir, 'mask')
    
    images, image_filenames = load_npy_files(images_dir)
    masks, mask_filenames = load_npy_files(masks_dir)
    
    # Assuming the number of images and masks are the same
    for i in range(len(images)):
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].imshow(images[i], cmap='gray')
        ax[0].set_title(f'Image: {image_filenames[i]}')
        ax[0].axis('off')
        
        ax[1].imshow(masks[i], cmap='gray')
        ax[1].set_title(f'Mask: {mask_filenames[i]}')
        ax[1].axis('off')
        
        plt.show()



In [ ]:
base_dir = 'D:\Kodingan\Kodingan TA\database'  # Modify this to your base directory
patient_number = 29 # Change this to display a different patient
patient_dir = os.path.join(base_dir, 'processed', f'patient{patient_number}')
display_images_and_masks(patient_dir)